In [12]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.snowball import FrenchStemmer
from dataprep.eda import *

In [2]:
fichiertest = "../../datasources/films/allocine_bigfoot_avis.csv"
test = pd.read_csv(fichiertest)

In [3]:
fichiertrain = "../../datasources/films/allocine_inception_avis.csv"
train = pd.read_csv(fichiertrain)

In [4]:
train.head()

,Note,Description,key
0,"5,0",après le chef doeuvre super héroïque the dark ...,0
1,"5,0",souvent l’on peut manquer d’inspiration ou de ...,1
2,"5,0",chef d’œuvre le film est absolument parfait ...,2
3,"3,5",le meilleur blockbuster de 2010 a pour thème l...,3
4,"5,0",un film aussi novateur que complexe dont la mi...,4


In [7]:
REMPLACE_SANS_ESPACE = re.compile("[;:!\'?,\"()\[\]]")
REMPLACE_AVEC_ESPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|[.]")
PUR_NOMBRE = re.compile("[0-9]")

def setClassBin(i):
    if (float(i.replace(',', '.')) > 3):
        return 1
    else:
        return 0
    
def preprocess(txt):
    txt = [PUR_NOMBRE.sub("", (str(line)).lower()) for line in txt] # retire les nombres (comme les années)
    txt = [line.replace('\n', ' ')  for line in txt] # Retire les \n (retours chariots)
    txt = [REMPLACE_SANS_ESPACE.sub("", line.lower()) for line in txt]
    txt = [REMPLACE_AVEC_ESPACE.sub(" ", line) for line in txt]
    return txt

def prepare_dataset(X):
    X['Description'] = pd.DataFrame(preprocess(X['Description']))
    french_stopwords = set(stopwords.words('french'))
    filtre_stopfr =  lambda text: [token for token in text if token.lower() not in french_stopwords]
    X['Description'] = [' '.join(filtre_stopfr(word_tokenize(item))) for item in X['Description']]
    stemmer = FrenchStemmer()
    X['Description'] = [stemmer.stem(w) for w in X['Description']]
    
    yList = [setClassBin(x) for x in X.Note]
    y = pd.DataFrame(yList)
    
    X = X.drop('Note', axis=1)
    return X, y

In [8]:
Xtrain, ytrain = prepare_dataset(train)